# Philippines Geo Data Cleaning Workbook

## Setup

In [1]:
import pandas as pd
import janitor
import math
%load_ext blackcellmagic

## Import and manage file that screens out non-ICM regions -- `raw_data/non_icm_loc.csv`

Breaking this out as a separate step so we can keep track of any changes made to the original file.

- On 04/14/2020, Batangas and Bulacan were changed from `remove == True` to `remove == Nan`
    - Details: prompted by Joy K., removed by Paul J.
- On 04/14/2020, First, Second, Third, and Fourth (districts of Manila) were added to the csv and set to `remove == TRUE`
    - Details: discovery by Paul J. in first round of EDA, confirmed by Joy K., removed by Paul J. 

In [2]:
# pulling in the non icm file to be used as a negative filer...
# and dropping any NaNs / NAs because I want this to be a pure negative filter...
# i.e. I will want this DF to be only regions I want to drop from my main df
non_icm_base_df = (
    pd.read_csv(filepath_or_buffer="raw_data/non_icm_loc.csv").dropna().clean_names()
)

# taking a peek at our data
non_icm_base_df.head()

,province,remove
0,Abra,True
6,Apayao,True
7,Aurora,True
9,Bataan,True
10,Batanes,True


## Create the SSOT file

In order to create our SSOT file, we need to take the `new_locations.csv` and remove all provinces labeled TRUE in the `non_icm_loc.csv` file, which will allow us to create an up to date file with all locations (ICM only). 

In [3]:
# pulling in the SSOT file for all new locations
new_locations_base_df = pd.read_csv(
    filepath_or_buffer="raw_data/new_locations.csv"
).clean_names()

# taking a peek at our data
new_locations_base_df.head()

,id,province,city,barangay,latitude,logitude
0,207538,Abra,Bangued,Agtangao,17.562700,120.637016
1,207472,Abra,Bangued,Angad,17.576710,120.621513
2,207374,Abra,Bangued,Bañacao,17.605690,120.595734
3,207385,Abra,Bangued,Bangbangar,17.606991,120.609749
4,207435,Abra,Bangued,Cabuloan,17.596331,120.612694


In [4]:
# removing all rows from `new_locations_base_df` that appear in `non_icm_base_df`
ssot_df = new_locations_base_df[
    (new_locations_base_df["province"].isin(non_icm_base_df["province"]) != True)
]

# capitalizing all needed fields to facilitate later matching
ssot_df = ssot_df.apply(
    lambda x: x.astype(str).str.upper() if (x.dtype == "object") else x
)

# getting rid of all whitespace just to be sure
ssot_df = ssot_df.apply(
    lambda x: x.astype(str).str.strip() if (x.dtype == "object") else x
)

# saving out the ssot_df for logging / future use purposes
ssot_df.to_csv(path_or_buf="processed_data/ssot_df.csv", index=False)

# inspecting the filtered data
ssot_df

,id,province,city,barangay,latitude,logitude
303,249388,AGUSAN DEL NORTE,BUENAVISTA,ABILAN,8.96710,125.451851
304,249466,AGUSAN DEL NORTE,BUENAVISTA,AGONG-ONG,8.96052,125.399017
305,249469,AGUSAN DEL NORTE,BUENAVISTA,ALUBIJID,8.92323,125.453773
306,249613,AGUSAN DEL NORTE,BUENAVISTA,GUINABSAN,8.84064,125.241096
307,249751,AGUSAN DEL NORTE,BUENAVISTA,LOWER OLAVE,8.79082,125.491669
...,...,...,...,...,...,...
41943,255227,ZAMBOANGA SIBUGAY,TUNGAWAN,TIGBANUANG,7.58116,122.430321
41944,255636,ZAMBOANGA SIBUGAY,TUNGAWAN,TIGBUCAY,7.54406,122.395416
41945,255647,ZAMBOANGA SIBUGAY,TUNGAWAN,TIGPALAY,7.47038,122.348328
41946,255194,ZAMBOANGA SIBUGAY,TUNGAWAN,TIMBABAUAN,7.61146,122.393120


## Clean up the *unclean* file -- `raw_data/original_locations.csv` -- and add new correct geo-mapping fields

Just as we did with the SSOT file, first we need to take the *unclean* file and remove all provinces labeled TRUE in the `non_icm_loc.csv` file, which will allow us to create an up to date file with all locations (ICM only). 

In [5]:
# pulling in our unclean data that needs to be matched to the geo data in the ssot_df...
# sorting by province just as the ssot df is, for convenience
unclean_base_df = (
    pd.read_csv(filepath_or_buffer="raw_data/original_locations.csv")
    .clean_names()
    .sort_values("province")
)

# taking a peek at our data
unclean_base_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population
41093,16,Cordillera Administrative Region,CAR,76,Abra,1510,Villaviciosa,38236,Tuquib,840
40878,16,Cordillera Administrative Region,CAR,76,Abra,1491,La Paz,38034,Toon,928
40879,16,Cordillera Administrative Region,CAR,76,Abra,1491,La Paz,38035,Udangan,490
40880,16,Cordillera Administrative Region,CAR,76,Abra,1492,Lacub,38036,Bacag,233
40881,16,Cordillera Administrative Region,CAR,76,Abra,1492,Lacub,38037,Buneg,827


In [6]:
# removing all rows from `unclean_base_df` that appear in `non_icm_base_df`
unclean_base_df = unclean_base_df[
    (unclean_base_df["province"].isin(non_icm_base_df["province"]) != True)
]

# taking a peek at our data
unclean_base_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population
37052,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34279,Rojales,2083
37047,14,CARAGA,Region XIII,66,Agusan del Norte,1295,City of Cabadbaran,34274,Mahaba,1250
37048,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34275,Cahayagan,2380
37049,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34276,Gosoon,1772
37050,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34277,Manoligao,1513


### Create `under_construction_df` -- a new go-forward DF which will be a copy of the *unclean* file with the new cleaned columns added

In [7]:
under_construction_df = unclean_base_df.copy()

### Create a new column -- `province_cleaned` -- to be appended to the `under_construction_df`, with the *correct* name for the province associated with each row:

First, create `province_mapping_df`, which will eventually serve as a mapping dictionnary of unclean to clean names, but will start by simply storing all unique province names in the *unclean* file.

In [8]:
# investigating unique values of province in the ssot data
unique_clean_provincename_list = [
    x.upper() for x in ssot_df["province"].unique().tolist()
]
print(unique_clean_provincename_list)

['AGUSAN DEL NORTE', 'AGUSAN DEL SUR', 'AKLAN', 'ALBAY', 'ANTIQUE', 'BASILAN', 'BATANGAS', 'BILIRAN', 'BOHOL', 'BUKIDNON', 'BULACAN', 'CAMIGUIN', 'CAPIZ', 'CATANDUANES', 'CEBU', 'COMPOSTELA VALLEY', 'DAVAO DEL NORTE', 'DAVAO DEL SUR', 'DAVAO ORIENTAL', 'DINAGAT ISLANDS', 'EASTERN SAMAR', 'GUIMARAS', 'ILOILO', 'LEYTE', 'MAGUINDANAO', 'MARINDUQUE', 'MASBATE', 'MISAMIS OCCIDENTAL', 'MISAMIS ORIENTAL', 'NEGROS OCCIDENTAL', 'NEGROS ORIENTAL', 'NORTH COTABATO', 'NORTHERN SAMAR', 'OCCIDENTAL MINDORO', 'ORIENTAL MINDORO', 'PALAWAN', 'ROMBLON', 'SAMAR', 'SARANGANI', 'SIQUIJOR', 'SORSOGON', 'SOUTH COTABATO', 'SOUTHERN LEYTE', 'SULTAN KUDARAT', 'SULU', 'SURIGAO DEL NORTE', 'SURIGAO DEL SUR', 'TAWI-TAWI', 'ZAMBOANGA DEL NORTE', 'ZAMBOANGA DEL SUR', 'ZAMBOANGA SIBUGAY']


In [9]:
# investigating unique values of region in the unclean data
unique_unclean_provincename_list = unclean_base_df["province"].unique().tolist()
print(unique_unclean_provincename_list)

['Agusan del Norte', 'Agusan del Sur', 'Aklan', 'Albay', 'Antique', 'Basilan', 'Batangas', 'Biliran', 'Bohol', 'Bukidnon', 'Bulacan', 'Camiguin', 'Capiz', 'Catanduanes', 'Cebu', 'City of Isabela (Capital)', 'Compostela Valley', 'Cotabato', 'Davao Del Sur', 'Davao Occidental', 'Davao Oriental', 'Davao del Norte', 'Dinagat Islands', 'Eastern Samar', 'Guimaras', 'Iloilo', 'Leyte', 'Maguindanao', 'Marinduque', 'Masbate', 'Misamis Occidental', 'Misamis Oriental', 'Negros Occidental', 'Negros Oriental', 'North Cotabato', 'Northern Samar', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon', 'Samar(Western Samar)', 'Sarangani', 'Siquijor', 'Sorsogon', 'South Cotabato', 'Southern Leyte', 'Sultan Kudarat', 'Sulu', 'Surigao del Norte', 'Surigao del Sur', 'Tawi-Tawi', 'Zamboanga Sibugay', 'Zamboanga del Norte', 'Zamboanga del Sur']


In [10]:
# make table for province name mapping possibiliies:

# make first column the unique set of all province names from the unclean data
province_mapping_df = pd.DataFrame(
    unique_unclean_provincename_list, columns=["unclean_provincename"]
)

# make second column the cast-to-upper version of the first collumn
province_mapping_df["upper_unclean_provincename"] = province_mapping_df[
    "unclean_provincename"
].str.upper()

# inspecting df we have so far
province_mapping_df.head()

,unclean_provincename,upper_unclean_provincename
0,Agusan del Norte,AGUSAN DEL NORTE
1,Agusan del Sur,AGUSAN DEL SUR
2,Aklan,AKLAN
3,Albay,ALBAY
4,Antique,ANTIQUE


Iterate over each unique province name in the *unclean* file, and check if the value in the `province` column matches a value contained in the `province` column of the SSOT file (accounting for capialization differences)

In [11]:
# create a function to loop through each unclean upper province name...
# if it finds that the unclean province name contains one of the clean names...
# set the value of our new column to be the clean name value...
# if it can't identify it, set the new value to "not yet matched"
def set_clean_province_name(row):
    # for each evaluation of each row passed, start off assuming no match
    province_match_found = False
    # loop over all unique clean province names
    for clean_province_name in unique_clean_provincename_list:
        # if we find a match in the clean province names...
        # meaning if we find that one of the clean province names...
        # occurs in the string of the unclean province name...
        # return the clean province name that was found
        if clean_province_name in row["upper_unclean_provincename"]:
            province_match_found = True
            return clean_province_name

    # if we struck out and didn't find a match, return a placeholder
    if not province_match_found:
        return "no_match_found"

# updating the df to include a new column for clean province name post matching
province_mapping_df = province_mapping_df.assign(
    matched_provincename=province_mapping_df.apply(set_clean_province_name, axis=1)
)

# inspecting the result
province_mapping_df.head()

,unclean_provincename,upper_unclean_provincename,matched_provincename
0,Agusan del Norte,AGUSAN DEL NORTE,AGUSAN DEL NORTE
1,Agusan del Sur,AGUSAN DEL SUR,AGUSAN DEL SUR
2,Aklan,AKLAN,AKLAN
3,Albay,ALBAY,ALBAY
4,Antique,ANTIQUE,ANTIQUE


In [12]:
# just inspecting the province names we couldn't match
province_names_failedtomatch_df = province_mapping_df.loc[
    province_mapping_df["matched_provincename"] == "no_match_found", :
]

province_names_failedtomatch_df

,unclean_provincename,upper_unclean_provincename,matched_provincename
15,City of Isabela (Capital),CITY OF ISABELA (CAPITAL),no_match_found
17,Cotabato,COTABATO,no_match_found
19,Davao Occidental,DAVAO OCCIDENTAL,no_match_found


In [13]:
# create a function for returning details on instances of "no_match_found" in a column

def get_no_match_found_details(df, col_name_with_matches):
    """
    Function to return match rate details for a column,
    where that column (when not matched) contains the string "no_match_found"
    """
    # first get number of total rows in df (which is # of province names)
    count_unique_province_names = len(df.index)

    # then get number of province names for which we found a match in our clean SSOT
    count_unmatched_province_names = (
        df[col_name_with_matches].str.count("no_match_found").sum()
    )

    # divide the number of matched province names by the total number of province names...
    # to get the percnt of province names we successfully matched
    province_names_match_rate = (
        count_unique_province_names - count_unmatched_province_names
    ) / count_unique_province_names

    # create string of details to output
    output_string = f"Out of {count_unique_province_names} unique province names in our data, we failed to match {count_unmatched_province_names}, resulting in a match rate of {'{:.2%}'.format(province_names_match_rate)}."

    return output_string

After having done the automated matches possible, perform the manual matching necessary based on additional research:

- For City of Isabela (Capital), the `matched_provincename` should be set to "BASILAN"
    - (Per Joy K.) City of Isabela (Capital) can be found under Basilan in NEW; we can hardcode the match. This differs from PSA SSOT (City of Isabela is officially under Region IX - PSA but NEW it is under AARM/Basilan).
- For Cotabato, the row in the `province_mapping_df` will be deleted, as the matching is more complicated and requires checking the city as well--i.e.--as there are multiple Cotabatos
    - (Per Joy K.) In this case we will hardcode the changes based on the corresponding Cities (Cotabato (City of Cotabato) >> Maguindanao (City of Cotabato) for all others change Cotabato >> North Cotabato)
        - PSA states there are officially 3 Cotabatos: North Cotabato, South Cotabato, and the City of Cotabato. In NEW we only have North Cotabato & South Cotabato. And in the OLD/ORG we have Cotabato, North Cotabato, and South Cotabato. 
        - In OLD/ORG Cotabato has municipalites called: City of Cotabato and the rest are all North Cotabatoian cities (confirmed via google). In NEW the City of Cotabato is under Maguindanao province. 
        - For future discrepencies I suggest reviewing the Municipality/Barangay levels to verify what to match the name(s) to. Let's document the change and what it should be according to the official PSA SSOT (e.g., City of Cotabato is under AARM/Maguindanao in NEW but in PSA it's SOCCSKSARGEN/City of Cotabato). 
- For Davao Occidental, the `matched_provincename` should be set to "DAVAO DEL SUR" 
    - (Per Joy K.) In this case it looks like in 2013 they created Davao Occidental from Davao del Sur which can be found in the NEW dataset. In this case we can again just manually match Davao Occidental >> Davao del Sur. 

In [14]:
# execute manual matching/fixes based on research above:

# manually matching City of Isabela (Capital)
province_mapping_df.loc[
    province_mapping_df["unclean_provincename"] == "City of Isabela (Capital)",
    "matched_provincename",
] = "BASILAN"

# removing Cotabato as its match depends on the city associated (i.e. logic more complicated)
province_mapping_df = province_mapping_df[
    province_mapping_df["unclean_provincename"] != "Cotabato"
]

# manually matching Davao Occidental
province_mapping_df.loc[
    province_mapping_df["unclean_provincename"] == "Davao Occidental",
    "matched_provincename",
] = "DAVAO DEL SUR"

In [15]:
# ensuring that our manual changes above cleared all the instance of "no_match_found"
get_no_match_found_details(
    df=province_mapping_df, col_name_with_matches="matched_provincename"
)

'Out of 53 unique province names in our data, we failed to match 0, resulting in a match rate of 100.00%.'

Use the `province_mapping_df` (and any other custom logic needed) to create the new `province_cleaned` column

- For any row where the `unclean_provincename` == "Cotabato", if the "city" listed is "Cotabato City", then set `province_cleaned` to "MAGUINDANAO"; otherwise, set `province_cleaned` to  "NORTH COTABATO"
- For all other rows (i.e. all rows where the `unclean_provincename` != "Cotabato"), look up the "province" name in our dictionnary table -- province_mapping_df -- and return the corresponding `matched_provincename`
- The algorithm also had trouble with parentheses like "SAMAR(WESTERN SAMAR)" and "CITY OF ISABELA (CAPITAL)" for some reason, so those were broken out manually as well.

In [16]:
# create a function to loop through all rows in our unclean base data...
# for each row where the unclean province name is NOT "Cotabato"...
# we'll rely on the `province_mapping_df`; then the custom logic for Cotabato
# is provided at the end
def create_province_cleaned_col(row):

    if "COTABATO" in row["province"].upper():
        if "Cotabato City" in row["city"]:
            return "MAGUINDANAO"
        else:
            return "NORTH COTABATO"
    elif "SAMAR(WESTERN SAMAR)" in row["province"].upper():
        return "SAMAR"
    elif "CITY OF ISABELA (CAPITAL)" in row["province"].upper():
        return "BASILAN"
    else:
        return (
            province_mapping_df.loc[
                province_mapping_df["upper_unclean_provincename"].str.contains(
                    row["province"].upper()
                ),
                "matched_provincename",
            ]
            .to_string(index=False)
            .strip()
        )


# updating the df to include a new column for clean province name post matching
under_construction_df = under_construction_df.assign(
    province_cleaned=under_construction_df.apply(create_province_cleaned_col, axis=1)
)

# inspecting the result
under_construction_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned
37052,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34279,Rojales,2083,AGUSAN DEL NORTE
37047,14,CARAGA,Region XIII,66,Agusan del Norte,1295,City of Cabadbaran,34274,Mahaba,1250,AGUSAN DEL NORTE
37048,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34275,Cahayagan,2380,AGUSAN DEL NORTE
37049,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34276,Gosoon,1772,AGUSAN DEL NORTE
37050,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34277,Manoligao,1513,AGUSAN DEL NORTE


### Write out `processed_data/under_construction_df.csv` to log the work done so far

In [17]:
# saving out the under construction df for logging
under_construction_df.to_csv(
    path_or_buf="processed_data/under_construction_df.csv", index=False
)

### City and Barangay are trickier and so we need to solve them all at once with more detailed analysis

Prior to jumping in, we'll prep by cleaning up some object formatting issues.

In [18]:
# capitalizing all needed fields to facilitate later matching
under_construction_df = under_construction_df.apply(
    lambda x: x.astype(str).str.upper() if (x.dtype == "object") else x
)

# getting rid of all whitespace just to be sure
under_construction_df = under_construction_df.apply(
    lambda x: x.astype(str).str.strip() if (x.dtype == "object") else x
)

# taking a look at the df
under_construction_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned
37052,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE
37047,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CITY OF CABADBARAN,34274,MAHABA,1250,AGUSAN DEL NORTE
37048,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE
37049,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE
37050,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE


First we go for all low-hanging fruit -- cities that we can match to the `ssot_df` because we can find an exact pairing between sets of province, city, and barangay between the `under_construction_df` df and the `ssot_df`. We'll perform this matching via a left join of the `ssot_df` onto the `under_construction_df`. We'll then flag all the rows that were matched successfully with this simple method. 

In [19]:
# performing the left merge
under_construction_df = pd.merge(
    under_construction_df,
    ssot_df,
    how="left",
    left_on=["province_cleaned", "city", "barangay"],
    right_on=["province", "city", "barangay"],
)

# if the match of all geo names after the merge was successful, flagging it with a 1...
# we can use the exisitance of a value in the 'id' column as a proxy for match, as...
# if there isn't a match, it'll be NaN
def flag_successful_full_matches(row):

    if math.isnan(row["id"]):
        return 0
    else:
        return 1


# adding the variable for successful flagging
under_construction_df = under_construction_df.assign(
    full_match_successful=under_construction_df.apply(
        flag_successful_full_matches, axis=1
    )
)

# cleaning up the result of the merge by removing not-yet-needed columns and prefixes
under_construction_df = under_construction_df.loc[
    :,
    [
        "region_id",
        "region",
        "region_alias",
        "province_id",
        "province_x",
        "city_id",
        "city",
        "barangay_id",
        "barangay",
        "population",
        "province_cleaned",
        "full_match_successful",
    ],
]
# rename the one column duplicated with prefix bc of the merge
under_construction_df.rename(columns={"province_x": "province"}, inplace=True)

# inspecting the df
under_construction_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CITY OF CABADBARAN,34274,MAHABA,1250,AGUSAN DEL NORTE,0
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1


Create a df with just the geo names we couldn't match to the `ssot_df` across all 3 geos so we can count the records still left to match. We'll do this multiple times from here on out until we arrive at 0 records we can't match.

In [20]:
# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 8525 rows where we couldn't find a full match."

(Round 1 of ad hoc research) Now let's make any fixes we noticed through ad hoc exploration and see how that affects our match rateNow let's make any fixes we noticed through ad hoc exploration and see how that affects our match rate.

In [21]:
# in Agusan Del Norte, in just_geo_names there is a city called "CITY OF CABADBARAN" that sould be "CABADBARAN CITY", per the ssot_df
under_construction_df.loc[
    (under_construction_df["province_cleaned"] == "AGUSAN DEL NORTE")
    & (under_construction_df["city"] == "CITY OF CABADBARAN"),
    ["city", "full_match_successful"],
] = ("CABADBARAN CITY", 1)

# in Agusan Del Sur, in just_geo_names there is a city called "CITY OF BAYUGA" that should be "BAYUGAN CITY", per the ssot_df
under_construction_df.loc[
    (under_construction_df["province_cleaned"] == "AGUSAN DEL SUR")
    & (under_construction_df["city"] == "CITY OF BAYUGAN"),
    ["city", "full_match_successful"],
] = ("BAYUGAN CITY", 1)

under_construction_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CABADBARAN CITY,34274,MAHABA,1250,AGUSAN DEL NORTE,1
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1


In [22]:
# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 8451 rows where we couldn't find a full match."

(Round 1 of ad hoc research)  It appears we've spotted one trend that can be corrected algorithmically -- we should look for instances of the city names that use the formulation "CITY OF xxxxxxx" and replace them with the formulation "xxxxxxx CITY".

In [23]:
# if we spot "CITY OF" in the particular record's 'city'...
# we'll set 'city' to the correct formulation -- "xxxxxxx CITY"...
# otherwise, we'll keep it as is
def invert_city_of_formulations(row):

    if "CITY OF" in row["city"]:
        return row["city"].partition("CITY OF ")[2] + " CITY"
    else:
        return row["city"]


# updating the df's values for city per the function above
under_construction_df = under_construction_df.assign(
    city=under_construction_df.apply(invert_city_of_formulations, axis=1)
)

# inspecting df
under_construction_df

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CABADBARAN CITY,34274,MAHABA,1250,AGUSAN DEL NORTE,1
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
28220,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29042,BAGANIAN,759,ZAMBOANGA DEL SUR,1
28221,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29043,BAYA-BAYA,1199,ZAMBOANGA DEL SUR,1
28222,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29044,CAPISAN,1738,ZAMBOANGA DEL SUR,1
28223,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1078,PITOGO,29147,MATIN-AO,7257,ZAMBOANGA DEL SUR,1


In [24]:
# using another merge back on the ssot_df to see how much that fix helped our match rate
under_construction_df = pd.merge(
    under_construction_df,
    ssot_df,
    how="left",
    left_on=["province_cleaned", "city", "barangay"],
    right_on=["province", "city", "barangay"],
)

# adding the variable for successful flagging
under_construction_df = under_construction_df.assign(
    full_match_successful=under_construction_df.apply(
        flag_successful_full_matches, axis=1
    )
)

# cleaning up the result of the merge by removing not-yet-needed columns and prefixes
under_construction_df = under_construction_df.loc[
    :,
    [
        "region_id",
        "region",
        "region_alias",
        "province_id",
        "province_x",
        "city_id",
        "city",
        "barangay_id",
        "barangay",
        "population",
        "province_cleaned",
        "full_match_successful",
    ],
]
# rename the one column duplicated with prefix bc of the merge
under_construction_df.rename(columns={"province_x": "province"}, inplace=True)

# inspecting the df
under_construction_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CABADBARAN CITY,34274,MAHABA,1250,AGUSAN DEL NORTE,1
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1


In [25]:
# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 7499 rows where we couldn't find a full match."

(Round 1 of ad hoc research) Looks like the formulation change from "CITY OF xxxxxxx" to "xxxxxxx CITY" fixed 952 -- (8451-7499) -- records!

(Round 2 of ad hoc research)  It appears we've spotted another trend that can be corrected algorithmically -- we should look for instances where the baranguy name doesn't match the SSOT because of the addition of the word or abbreviation for "population" -- poblacion.

In [26]:
# if we spot "(POB.)" in any of the baranguys...
# delete that substrint from that row

def remove_poblacion_from_baranguy(row):

    if "(POB.)" in row["barangay"]:
        return row["barangay"].replace("(POB.)", "").strip()
    elif "POB." in row["barangay"]:
        return row["barangay"].replace("POB.", "").strip()
    else:
        return row["barangay"]


# updating the df's values for city per the function above
under_construction_df = under_construction_df.assign(
    barangay=under_construction_df.apply(remove_poblacion_from_baranguy, axis=1)
)

# inspecting df
under_construction_df

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CABADBARAN CITY,34274,MAHABA,1250,AGUSAN DEL NORTE,1
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
28220,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29042,BAGANIAN,759,ZAMBOANGA DEL SUR,1
28221,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29043,BAYA-BAYA,1199,ZAMBOANGA DEL SUR,1
28222,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29044,CAPISAN,1738,ZAMBOANGA DEL SUR,1
28223,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1078,PITOGO,29147,MATIN-AO,7257,ZAMBOANGA DEL SUR,1


In [27]:
# standard code used to re-check how many un-matched records remain...
# using another merge back on the ssot_df to see how much that fix helped our match rate
under_construction_df = pd.merge(
    under_construction_df,
    ssot_df,
    how="left",
    left_on=["province_cleaned", "city", "barangay"],
    right_on=["province", "city", "barangay"],
)

# adding the variable for successful flagging
under_construction_df = under_construction_df.assign(
    full_match_successful=under_construction_df.apply(
        flag_successful_full_matches, axis=1
    )
)

# cleaning up the result of the merge by removing not-yet-needed columns and prefixes
under_construction_df = under_construction_df.loc[
    :,
    [
        "region_id",
        "region",
        "region_alias",
        "province_id",
        "province_x",
        "city_id",
        "city",
        "barangay_id",
        "barangay",
        "population",
        "province_cleaned",
        "full_match_successful",
    ],
]
# rename the one column duplicated with prefix bc of the merge
under_construction_df.rename(columns={"province_x": "province"}, inplace=True)

# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 6124 rows where we couldn't find a full match."

(Round 2 of ad hoc research) Looks like the change to strip all barangays of the (POB.) string fixed 1,375 -- (7499-6124) -- records.

(Round 3 of ad hoc research) It appears we've spotted another trend that can be corrected algorithmically -- we should look for instances where the baranguy name doesn't match the SSOT because the barangay name is "empty" and delete the row.

In [28]:
# show that "empty" is actually the 3rd most prevalent barangay name
under_construction_df["barangay"].value_counts(dropna=False)

POBLACION      530
SAN ISIDRO     220
EMPTY          203
SAN JOSE       163
SAN ROQUE      128
              ... 
DATU BALONG      1
LAMUD            1
PAMARAWAN        1
KALANAWE I       1
NAPNUD           1
Name: barangay, Length: 16325, dtype: int64

In [29]:
# removing any rows from `under_construction_df` if the barangay name is 'EMPTY'
under_construction_df = under_construction_df[
    under_construction_df["barangay"] != "EMPTY"
]

In [30]:
# standard code used to re-check how many un-matched records remain...
# using another merge back on the ssot_df to see how much that fix helped our match rate
under_construction_df = pd.merge(
    under_construction_df,
    ssot_df,
    how="left",
    left_on=["province_cleaned", "city", "barangay"],
    right_on=["province", "city", "barangay"],
)

# adding the variable for successful flagging
under_construction_df = under_construction_df.assign(
    full_match_successful=under_construction_df.apply(
        flag_successful_full_matches, axis=1
    )
)

# cleaning up the result of the merge by removing not-yet-needed columns and prefixes
under_construction_df = under_construction_df.loc[
    :,
    [
        "region_id",
        "region",
        "region_alias",
        "province_id",
        "province_x",
        "city_id",
        "city",
        "barangay_id",
        "barangay",
        "population",
        "province_cleaned",
        "full_match_successful",
    ],
]
# rename the one column duplicated with prefix bc of the merge
under_construction_df.rename(columns={"province_x": "province"}, inplace=True)

# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 5921 rows where we couldn't find a full match."

(Round 3 of ad hoc research) Looks like the change to delete all rows where the "barangay" value was "EMPTY" fixed 203 -- (6124-5921) -- records.

(Round 4 of ad hoc research) It appears that my previous logic for cleaning up province names failed slightly, as it didn't account for instances of duplication (i.e. it tagged the province name of "LEYTE\n LEYTE" as correct because it does CONTAIN "LEYTE". It should be a quick fix to manually remove these instances. 

In [31]:
# manually correcting all instances of the wrong "province_cleaned" name due to the duplicated "LEYTE"
under_construction_df.loc[
    under_construction_df["province_cleaned"] == "LEYTE\n LEYTE", "province_cleaned"
] = "LEYTE"

In [32]:
# standard code used to re-check how many un-matched records remain...
# using another merge back on the ssot_df to see how much that fix helped our match rate
under_construction_df = pd.merge(
    under_construction_df,
    ssot_df,
    how="left",
    left_on=["province_cleaned", "city", "barangay"],
    right_on=["province", "city", "barangay"],
)

# adding the variable for successful flagging
under_construction_df = under_construction_df.assign(
    full_match_successful=under_construction_df.apply(
        flag_successful_full_matches, axis=1
    )
)

# cleaning up the result of the merge by removing not-yet-needed columns and prefixes
under_construction_df = under_construction_df.loc[
    :,
    [
        "region_id",
        "region",
        "region_alias",
        "province_id",
        "province_x",
        "city_id",
        "city",
        "barangay_id",
        "barangay",
        "population",
        "province_cleaned",
        "full_match_successful",
    ],
]
# rename the one column duplicated with prefix bc of the merge
under_construction_df.rename(columns={"province_x": "province"}, inplace=True)

# checking number of yet-to-be-matched geo names
problematic_geo_names = under_construction_df.loc[
    under_construction_df["full_match_successful"] == 0, :
]
f"There are {len(problematic_geo_names)} rows where we couldn't find a full match."

"There are 4422 rows where we couldn't find a full match."

(Round 4 of ad hoc research) Looks like the change to remedy the duplicated "LEYTE" cleaned province names fixed 1,499 -- (5921-4422) -- records.

In [33]:
# taking at look at the df as it stands after 4 ad hoc research-driven updates
under_construction_df

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
0,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34279,ROJALES,2083,AGUSAN DEL NORTE,1
1,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1295,CABADBARAN CITY,34274,MAHABA,1250,AGUSAN DEL NORTE,1
2,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34275,CAHAYAGAN,2380,AGUSAN DEL NORTE,1
3,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34276,GOSOON,1772,AGUSAN DEL NORTE,1
4,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34277,MANOLIGAO,1513,AGUSAN DEL NORTE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
28017,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29042,BAGANIAN,759,ZAMBOANGA DEL SUR,1
28018,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29043,BAYA-BAYA,1199,ZAMBOANGA DEL SUR,1
28019,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29044,CAPISAN,1738,ZAMBOANGA DEL SUR,1
28020,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1078,PITOGO,29147,MATIN-AO,7257,ZAMBOANGA DEL SUR,1


### Write out `processed_data/under_construction_df.csv` to log the work done so far

In [34]:
# saving out the under construction df for logging -- this is the most up-to-date file
under_construction_df.to_csv(
    path_or_buf="processed_data/under_construction_df.csv", index=False
)

# saving out the problematic geography names file as well, for logging purposes
problematic_geo_names.to_csv(
    path_or_buf="processed_data/problematic_geo_names.csv", index=False
)

(Round 5 of ad hoc research) ... whatever I discover next in digging into the `problematic_geo_names` df to identify trends in mismatches between the `under_construction_df` and the `ssot_df` will go here.

In [35]:
# looking again into the geographies I still can't match to the SSOT df to look for trends
problematic_geo_names

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population,province_cleaned,full_match_successful
5,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,34278,POBLACION (CARMEN),5507,AGUSAN DEL NORTE,0
22,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,43371,NUEVA FUERZA,0,AGUSAN DEL NORTE,0
30,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1297,JABONGA,34285,A. BELTRAN (CAMALIG),1149,AGUSAN DEL NORTE,0
33,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,45136,LA PAZ,0,AGUSAN DEL NORTE,0
34,14,CARAGA,REGION XIII,66,AGUSAN DEL NORTE,1296,CARMEN,44967,GUADALUPE,0,AGUSAN DEL NORTE,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27891,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1071,SAN MIGUEL,44314,MAHAYAG,0,ZAMBOANGA DEL SUR,0
27893,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1071,SAN MIGUEL,44882,SAN JOSE,0,ZAMBOANGA DEL SUR,0
27902,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1081,GUIPOS,29195,POBLACION (GUIPOS),1032,ZAMBOANGA DEL SUR,0
27976,10,ZAMBOANGA PENINSULA,REGION IX,48,ZAMBOANGA DEL SUR,1073,TABINA,29055,DONA JOSEFINA,807,ZAMBOANGA DEL SUR,0


In [36]:
# ssot_df.loc[
#     (ssot_df["province"] == "ZAMBOANGA DEL NORTE") & (ssot_df["city"] == "DAPITAN CITY"), :
# ].head()

### Next Steps:

- Start on round 5 of ad hoc research -- looking for more trends that we can exploit to successfully match the outstanding unmatched 4422 pairings of province, city, and barangay.
- See what impact of accents may be (like SANTO NIÑO for example). It would make things a little easier if we could get rid of those entirely, but that's probably not doable I imagine.

### Questions / Roadblocks (for Joy):
- So right now, where we're at is that the `under_construction_df` represents all the work I've done to-date. If the "full_match_successful" column is == 1, then I have been able to match the 3-part-key (province/city/barangay) EXACTLY to the SSOT file. Of the 28,022 rows in the initial unclean file, there are only 4,422 pairings that I can't match exactly, which I have stored in the `problematic_geo_names` df. Both of these files are available in .csv format in the `processed_data` subfolder, so what I could always appreciate your help with is looking through the `problematic_geo_names` file and trying to see if you could spot patterns / issues (like the ones I've spotted above in the various ad hoc research steps) that would help me to keep reducing that 4,422 number. Right now I bet the next ones will have to do with the occurences of "POBLACION" and parentheses in the "barangay" column. Also, accents are giving me trouble for sure, so I'll need to think through how to handle that. Feel free to let me know if you do / don't care that the accents are captured in the final names. 
- Is there a way you could grant me admin access over the repo? I'd like to look into setting up a few things like Milestones, Projects, and GitHub actions. 